# INITIALIZATION

In [7]:
# Generator
import networkx as nx
import random

def generate_random_graph(num_nodes, num_edges, graph_type="random"):
    """
    Generates a graph with random edges.
    :param num_nodes: Number of nodes in the graph
    :param num_edges: Number of edges in the graph
    :param graph_type: Type of graph ("random", "complete", "cycle", etc.)
    :return: A NetworkX graph
    """
    if graph_type == "random":
        G = nx.gnm_random_graph(num_nodes, num_edges)
    elif graph_type == "complete":
        G = nx.complete_graph(num_nodes)
    elif graph_type == "cycle":
        G = nx.cycle_graph(num_nodes)
    elif graph_type == "star":
        G = nx.star_graph(num_nodes - 1)  # Subtract 1 for center node
    else:
        raise ValueError(f"Unknown graph type: {graph_type}")
    
    return G

def save_graph_to_file(graph, filename="graph.txt"):
    """
    Saves the graph's edges to a file in the format:
    node1 node2
    :param graph: A NetworkX graph
    :param filename: Output filename
    """
    with open(filename, 'w') as f:
        for u, v in graph.edges():
            f.write(f"{u + 1} {v + 1}\n")  # Writing 1-indexed nodes

# Example Usage:
num_nodes = 500  # Number of nodes
num_edges = 10000  # Number of edges
graph_type = "random"  # Can be "random", "complete", "cycle", or "star"
G = generate_random_graph(num_nodes, num_edges, graph_type)
save_graph_to_file(G, "syn_random.mtx")
graph_type = "complete"  # Can be "random", "complete", "cycle", or "star"
G = generate_random_graph(num_nodes, num_edges, graph_type)
save_graph_to_file(G, "syn_complete.mtx")
graph_type = "cycle"  # Can be "random", "complete", "cycle", or "star"
G = generate_random_graph(num_nodes, num_edges, graph_type)
save_graph_to_file(G, "syn_cycle.mtx")
graph_type = "star"  # Can be "random", "complete", "cycle", or "star"
G = generate_random_graph(num_nodes, num_edges, graph_type)
save_graph_to_file(G, "syn_star.mtx")

In [30]:
# Set max length
import numpy as np
import random
import networkx as nx
import csrgraph as cg
def max_random_walk_length(graph: nx.Graph, T, gamma):
    """Calculate the maximum length of random walks to achieve a desired ratio of invalid walks."""
    n = graph.number_of_nodes()
    m = graph.number_of_edges()
    d = np.array([deg for node, deg in dict(graph.degree()).items() if node != list(T)[0]])
    d_norm = np.linalg.norm(d)

    A = nx.to_numpy_array(graph)
    A = np.delete(A, list(T)[0], axis=0)
    A = np.delete(A, list(T)[0], axis=1)

    spectral_radius = max(abs(np.linalg.eigvals(A)))

    max_length = int((np.log(m * gamma / (np.sqrt(n - 1))) * d_norm / np.log(spectral_radius)))
    return max_length

def random_walk(csr_graph, start_node, max_length, target_set):
    """
    Perform a random walk on a CSR graph.
    
    Parameters:
        csr_graph (cg.csrgraph): The csrgraph object.
        start_node (int): The starting node of the random walk.
        max_length (int): Maximum length of the walk.
        target_set (set): A set of target nodes that stops the walk when reached.

    Returns:
        tuple: The path of nodes visited in the walk and the final position.
    """
    walk = [start_node]
    current_node = start_node
    position = 0
    
    # Access the CSR matrix directly
    csr_matrix = csr_graph.mat

    if current_node in target_set:
        return ([start_node], position)
    
    for _ in range(max_length):
        position += 1
        
        # Get neighbors of the current node using CSR matrix attributes
        start_index = csr_matrix.indptr[current_node]
        end_index = csr_matrix.indptr[current_node + 1]
        neighbors = csr_matrix.indices[start_index:end_index]
        
        # Filter out nodes already in the walk to avoid revisiting
        neighbors = list(set(neighbors) - set(walk))
        
        # Stop if no unvisited neighbors or if current node is in target set
        if len(neighbors) == 0 or current_node in target_set:
            break
        
        # Choose the next node randomly from available neighbors
        next_node = random.choice(neighbors)
        walk.append(next_node)
        current_node = next_node
    
    return walk, position

def effective_resistance_approximation(graph: nx.Graph, T, Q, max_length, epsilon):
    n = len(graph)  # Number of nodes in the graph
    rho = int(0.001 * np.log(n) / epsilon**2)  # Number of random walks for accuracy

    C = np.zeros(n)  # Effective resistance approximation array
    R = np.zeros(n)  # Effective resistance results for nodes in Q
    W = []

    # Step 2: Perform random walks for each edge
    for (i, j) in graph.edges():
        for _ in range(rho):
            # Generate two random walks from endpoints of the edge
            w1, pos1 = random_walk(graph, i, max_length, T)
            w2, pos2 = random_walk(graph, j, max_length, T)
            # If both walks reach the target set, combine and add to W
            if w1[-1] in T and w2[-1] in T:
                W.append((w1, pos1))
                W.append((w2, pos2))
            
    # Step 8-12: Update C array based on the combined walks in W
    for walk, total_length in W:
        for u in walk:
            if u in Q:  # Only update for nodes in Q
                C[u] += 1 / (rho * total_length)  # Update C[u] for effective resistance

    # Step 13-14: Calculate effective resistance by inverting C
    for i, q in enumerate(Q):
        if C[q] != 0:
            R[i] = 1 / C[q]
        else:
            R[i] = np.inf  # If C[q] is 0, effective resistance is considered infinite

    return R, W

# Example

In [ ]:
# Example usage
G = nx.Graph()
nodes = [0, 1, 2, 3]
edges = [(0, 1), (0, 2), (0, 3), (1, 3), (2, 3)]
G.add_nodes_from(nodes)
G.add_edges_from(edges)

# Define parameters
T = {0}  # target node
Q = {1, 2, 3}  # Query nodes (excluding v)
gamma = 0.95
max_length = max_random_walk_length(G, T, gamma) # Maximum length for random walks
epsilon = 0.005  # Desired accuracy

# Run the effective resistance approximation
R, W = effective_resistance_approximation(G, T, Q, max_length, epsilon)
print("Improved Approximate Effective Resistance:")
for idx, q in enumerate(Q):
    for i in T:
        print(f"R_{i}{q} ≈ {R[idx]}")

# DELETE EDGE

In [5]:
import networkx as nx
import numpy as np

def delete_edge(graph: nx.Graph, W, v, Q, epsilon, P=[]):
    # List to store results for each edge removed
    removed_edges_list = []
    n = graph.number_of_nodes()
    rho = int(np.log(n) / epsilon**2)  # Number of random walks for accuracy

    # Iterate over each edge in the graph
    for (x, y) in graph.edges():
        # Create a new graph without the edge (x, y)
        graph_without_edge = graph.copy()
        graph_without_edge.remove_edge(x, y)
        
        # Check if removing the edge disconnects the graph
        if not nx.is_connected(graph_without_edge):
            removed_edges_list.append(((x, y), 0))
            continue
        
        # Initialize effective resistance value for this edge removal
        resistance_sum = 0

        # Filter walks that contain nodes x or y
        walks_containing_x_or_y = [(walk, total_length) for walk, total_length in W if x in walk or y in walk]

        # Compute effective resistance using each walk that includes x or y
        for walk_data in walks_containing_x_or_y:
            if isinstance(walk_data, tuple) and len(walk_data) == 2:
                walk, total_length = walk_data
                for u in walk:
                    if u in Q:
                        # Update resistance values according to walk length and target nodes
                        resistance_sum += n / (rho * total_length)
        
        # Append this edge and its resistance contribution to the result list
        removed_edges_list.append(((x, y), resistance_sum))

    return removed_edges_list

# APPROXISC

In [6]:
def APPROXISC(G:nx.Graph, k, v, epsilon):
    P = {}
    T = {v}
    Q = G.copy(G)
    Q.remove_node(v)
    gamma = 0.95
    for i in range(k):
        max_length = max_random_walk_length(G, T, gamma) # Maximum length for random walks
        R, W = effective_resistance_approximation(G, T, Q, max_length, epsilon)
        removed_edges_list = delete_edge(G, W, v, Q, epsilon)
        arg_max = max(removed_edges_list)
        P[i] = arg_max[0]
        G.remove_edge(arg_max[0][0], arg_max[0][1])
    return P

# Example

In [7]:
G = nx.Graph()
nodes = [0, 1, 2, 3, 4]
edges = [(0, 2), (4, 2), (1, 2), (1, 3), (2, 3), (1, 4)]
G.add_nodes_from(nodes)
G.add_edges_from(edges)
# Parameters
k = 2  # Need remove edges
v = 2  # Starting node in T
epsilon = 0.05  # Epsilon parameter
P = APPROXISC(G, k, v, epsilon)

In [8]:
print(f"Top-k edges need to remove {P}")

Top-k edges need to remove {0: (2, 4), 1: (2, 3)}


# Input Data

In [23]:
import scipy.sparse as sp
# Initialize an empty list to store the tuples
result = []
# Open and read the file
with open('./hamster.mtx', 'r') as file:
    for line in file:
        # Split each line into two integers
        a, b = map(int, line.split())
        
        # Adjust to 0-based indexing and create the tuple
        result.append((a - 1, b - 1))

# Input to graph
G = nx.Graph()
G.add_edges_from(result)
adj_matrix = sp.csr_matrix(nx.adjacency_matrix(G))
# Use the CSR matrix with cg.csrgraph
graph_csr = cg.csrgraph(adj_matrix)

# Optional: Print to verify
print("CSR graph initialized:", graph_csr)

CSR graph initialized: <csrgraph.graph.csrgraph object at 0x000001F5D5801BB0>


In [27]:
import csrgraph as cg
import numpy as np

def random_walk_to_target(csr_graph, start_node, target_node, max_steps=100):
    current_node = start_node
    path = [current_node]
    csr_matrix = csr_graph.mat  # Access the CSR matrix directly

    for _ in range(max_steps):
        # Check if we have reached the target node
        if current_node == target_node:
            break

        # Get neighbors of the current node using CSR matrix attributes
        start_index = csr_matrix.indptr[current_node]
        end_index = csr_matrix.indptr[current_node + 1]
        neighbors = csr_matrix.indices[start_index:end_index]

        # If no neighbors, stop the walk
        if len(neighbors) == 0:
            break

        # Randomly select the next node from neighbors
        current_node = np.random.choice(neighbors)
        path.append(current_node)

    return path

# Example usage
# Initialize the graph from your adjacency matrix (assuming `adj_matrix` is ready)
csr_graph = cg.csrgraph(adj_matrix)

# Define the start and target nodes
start_node = 0      # Adjust to the desired starting node
target_node = 10    # Adjust to the desired target node

# Run the random walk
path = random_walk_to_target(csr_graph, start_node, target_node)
print("Random walk path:", path)


Random walk path: [0, np.int32(9), np.int32(17), np.int32(15), np.int32(2), np.int32(43), np.int32(606), np.int32(325), np.int32(319), np.int32(469), np.int32(670), np.int32(469), np.int32(400), np.int32(463), np.int32(479), np.int32(56), np.int32(1065), np.int32(38), np.int32(39), np.int32(729), np.int32(1137), np.int32(1197), np.int32(817), np.int32(1582), np.int32(1579), np.int32(1583), np.int32(820), np.int32(1580), np.int32(10)]


In [29]:
adj_matrix.toarray()

array([[0, 1, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0]])

In [ ]:
P = APPROXISC(G, k, v, epsilon)

In [11]:
print(f"Top-k edges need to remove {P}")

Top-k edges need to remove {0: (103, 104), 1: (102, 95)}
